In [1]:
%load_ext autoreload
%autoreload 2
from CellTracker.test_matching_models import match_by_fpm, rotation_align_by_fpm,\
    load_fpm
from CellTracker.fpm import FlexiblePointMatcherConv, FlexiblePointMatcherOriginal
from CellTracker.trackerlite import plot_initial_matching
import numpy as np

In [2]:
def cal_acc(points2, pairs):
    ids = points2[:, 3]>-1
    ids_ = np.nonzero(ids)[0]
    print(ids_+1)
    groundtruth = points2[ids, 3]
    pairs_ids = np.asarray([(points1[row[0], 3], points2[row[1],3]) for row in pairs if row[1] in (ids_+1).tolist()])
    acc = np.count_nonzero(np.abs(pairs_ids[:, 0]-pairs_ids[:,1])<0.1)/groundtruth.size
    return acc

match_model = FlexiblePointMatcherConv(num_skip=2)
fpm_model = load_fpm(fpm_model_path="./fpm_models/fpm_conv_2skip_worm3_180_deg.h5", match_model=match_model)

2023-10-04 10:26:03.329353: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-04 10:26:03.332809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-04 10:26:03.333231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-04 10:26:03.333984: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [34]:
print(points2)

[[-5.30407384e-01  5.62253349e-03 -2.42868805e-01 -1.00000000e+00]
 [-4.71781410e-01  2.30956518e-02 -2.17106926e-01 -1.00000000e+00]
 [-4.10257984e-01  2.26054557e-02 -2.17106926e-01  4.50000000e+01]
 [-3.66718252e-01  2.58149500e-02 -2.42868805e-01 -1.00000000e+00]
 [-5.37367816e-01  1.11356890e-02 -1.91630761e-01  8.00000000e+01]
 [-2.64639371e-02 -1.46534725e-01 -1.41083192e-01  4.10000000e+01]
 [ 6.58925735e-02 -1.33022234e-01 -1.41083192e-01  1.00000000e+01]
 [-5.01260122e-01 -1.18171855e-01 -1.41083192e-01  3.60000000e+01]
 [-3.98965050e-01  3.19504575e-02 -1.66940309e-01 -1.00000000e+00]
 [ 1.34885925e-01 -3.56947607e-02 -1.41083192e-01  7.00000000e+00]
 [ 7.45980471e-02 -3.98500847e-02 -1.15702265e-01 -1.00000000e+00]
 [-5.58124288e-02 -2.15388145e-01 -1.15702265e-01  5.80000000e+01]
 [-4.74170163e-02 -8.77755603e-02 -1.15702265e-01  2.10000000e+01]
 [ 3.31784071e-02 -8.71783208e-02 -1.15702265e-01  1.30000000e+01]
 [-9.64989541e-02 -5.73429447e-02 -1.15702265e-01  0.00000000e

In [18]:
match_model = FlexiblePointMatcherConv(num_skip=2)
fpm_model = load_fpm(fpm_model_path="./fpm_models/fpm_conv_2skip_worm3_180_deg.h5", match_model=match_model)
import glob
npy_files = glob.glob("fDNC_Data/test_tracking/real_jeff_all" + "/*.npy")
accs = []
for i in range(100):
    points1 = np.load(npy_files[i])
    points2 = np.load(npy_files[i+200])
    pairs = match_by_fpm(fpm_model, points1=points1[:, :3], points2=points2[:, :3],
                ids_ref=points1[:, 3], ids_tgt=points2[:, 3], match_method="greedy")
    accs.append(cal_acc(points2, pairs))
print(f"average acc = {np.mean(accs)}, std={np.std(accs)}")

average acc = 0.7475656832590566, std=0.1578968397968092


In [30]:
match_model = FlexiblePointMatcherConv(num_skip=2)
fpm_model = load_fpm(fpm_model_path="./fpm_models/fpm_conv_2skip_worm3_180_deg.h5", match_model=match_model)
match_model2 = FlexiblePointMatcherConv(num_skip=2)
fpm_model_2 = load_fpm(fpm_model_path="./fpm_models/fpm_conv_2skip_worm3_10_deg.h5", match_model=match_model2)

import glob
npy_files = glob.glob("fDNC_Data/test_tracking/real_jeff_all" + "/*.npy")
accs = []
for i in range(100):
    points1 = np.load(npy_files[i])
    points2 = np.load(npy_files[i+200])
    aligned_t1, norm_t2, pairs = rotation_align_by_fpm(fpm_model_rot, points1=points1[:, :3], 
                                                                  points2=points2[:, :3],
                ids_ref=points1[:, 3], ids_tgt=points2[:, 3], match_method="greedy")

    pairs_2 = match_by_fpm(fpm_model_2, aligned_t1, norm_t2,
                ids_ref=points1[:, 3], ids_tgt=points2[:, 3], match_method="greedy")

    accs.append(cal_acc(points2, pairs_2))
print(f"average acc = {np.mean(accs)}, std={np.std(accs)}")

average acc = 1.0089915546491581, std=0.2099461835550842


In [46]:
match_model = FlexiblePointMatcherConv(num_skip=2)
fpm_model_rot = load_fpm(fpm_model_path="./fpm_models/fpm_conv_2skip_worm3_180_deg.h5", match_model=match_model)

import glob
npy_files = glob.glob("fDNC_Data/test_tracking/real_jeff_all" + "/*.npy")
points1 = np.load(npy_files[0])
points2 = np.load(npy_files[100])
aligned_t1, norm_t2, pairs = rotation_align_by_fpm(fpm_model_rot, points1=points1[:, :3], 
                                                              points2=points2[:, :3],
            ids_ref=points1[:, 3], ids_tgt=points2[:, 3], match_method="greedy")

cal_acc(points2, pairs)

[ 1  3  4  5  6  7 10 11 12 14 16 17 18 19 20 22 23 24 25 26 27 28 29 30
 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 52 57 59 60 61 64 65
 66 67 69 72 73 77 78 80 82 83 89 90 91 94 95 98]


0.453125

In [47]:
match_model2 = FlexiblePointMatcherConv(num_skip=2)
fpm_model = load_fpm(fpm_model_path="./fpm_models/fpm_conv_2skip_worm3_10_deg.h5", match_model=match_model2)
pairs = match_by_fpm(fpm_model, aligned_t1, norm_t2,
            ids_ref=points1[:, 3], ids_tgt=points2[:, 3], match_method="greedy")

cal_acc(points2, pairs)

[ 1  3  4  5  6  7 10 11 12 14 16 17 18 19 20 22 23 24 25 26 27 28 29 30
 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 52 57 59 60 61 64 65
 66 67 69 72 73 77 78 80 82 83 89 90 91 94 95 98]


0.65625

In [48]:
match_model = FlexiblePointMatcherOriginal()
fpm_model = load_fpm(fpm_model_path="./fpm_models/fpm_ori_worm3_180_deg.h5", match_model=match_model)
import glob
npy_files = glob.glob("fDNC_Data/test_tracking/real_jeff_all" + "/*.npy")
points1 = np.load(npy_files[0])
points2 = np.load(npy_files[100])
pairs = match_by_fpm(fpm_model, points1=points1[:, :3], points2=points2[:, :3],
            ids_ref=points1[:, 3], ids_tgt=points2[:, 3], match_method="greedy")
cal_acc(points2, pairs)

[ 1  3  4  5  6  7 10 11 12 14 16 17 18 19 20 22 23 24 25 26 27 28 29 30
 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 52 57 59 60 61 64 65
 66 67 69 72 73 77 78 80 82 83 89 90 91 94 95 98]


0.34375